# PipeLine

**Menghubungkan google colab dengan google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Instalasi Library**
*   menginstall beberapa library yang dibutuhkan



In [ ]:
# !pip install mysqlclient
# !pip install mysql
!pip install mysql-connector-python
# !pip install pymysql
# !pip install pyodbc
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import psycopg2
import pandas as pd
import mysql.connector

**Mengambil data Iris dari SQL Server**
*   Melakukan load data yang telah ada di gdrive
*   Melakukan select semua data pada tabel iris
*   Pada tabel di database ini terdapat kolom id, petalwidth, dan class
*   Lalu menampilkan semua data


In [ ]:
base = '/content/drive/MyDrive/prosaindata/tugas/Dataset/iris_SqlServer.csv'
def load_iris_SqlServer():
  data = pd.read_csv(base)
  return data

In [ ]:
df_sqlServer=load_iris_SqlServer()
df_sqlServer= df_sqlServer.drop('ID ', axis=1)
df_sqlServer

,petalwidth,class
0,0.2,Iris-setosa
1,0.2,Iris-setosa
2,0.2,Iris-setosa
3,0.2,Iris-setosa
4,0.2,Iris-setosa
...,...,...
145,2.3,Iris-virginica
146,1.9,Iris-virginica
147,2.0,Iris-virginica
148,2.3,Iris-virginica


**Mengambil data Iris dari Elephant**
*   Mendefine engine yang nantinya akan terhubung dengan database di Elephant
*   Melakukan select semua data pada tabel iris
*   Pada tabel di database ini terdapat kolom id dan sepalwidth
*   Lalu menampilkan data menggunakan dataframe

In [ ]:
# membuat parameter dengan dictionary
param_dic = {
    "host"      : "tiny.db.elephantsql.com",
    "database"  : "qkublreo",
    "user"      : "qkublreo",
    "password"  : "f0M9bywbpfWpuxNOCxC9N0DJHVDgkE8e"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

# Connect to the database
conn = connect(param_dic)

column_names = ["ID","sepalwidth"]

df_elephant = postgresql_to_dataframe(conn, "select * from iris", column_names)
df_elephant = df_elephant= df_elephant.drop('ID', axis=1)
df_elephant

Connecting to the PostgreSQL database...
Connection successful


,sepalwidth
0,3.5
1,3.0
2,3.2
3,3.1
4,3.6
...,...
145,3.0
146,2.5
147,3.0
148,3.4


**Mengambil data Iris dari PostgreSQL Lokal**
*   Melakukan load data iris pada gdrive
*   Melakukan select semua data pada tabel iris
*   Pada tabel di database ini terdapat kolom id dan sepallength
*   Lalu menampilkan data menggunakan dataframe

In [ ]:
base1 = '/content/drive/MyDrive/prosaindata/tugas/Dataset/iris_Postgres.csv'
def load_iris_PostgreSQL():
  data = pd.read_csv(base1)
  return data

In [ ]:
df_postgreSQL=load_iris_PostgreSQL()
df_postgreSQL = df_postgreSQL= df_postgreSQL.drop('ID ', axis=1)
df_postgreSQL

,sepallength
0,5.1
1,4.9
2,4.7
3,4.6
4,5.0
...,...
145,6.7
146,6.3
147,6.5
148,6.2


**Mengambil data Iris dari MySQL**
*   Mendefine engine yang nantinya akan terhubung dengan database di MySQL
*   Melakukan select semua data pada tabel iris
*   Pada tabel di database ini terdapat kolom id dan petallength
*   Lalu menampilkan data menggunakan dataframe

In [ ]:
mydb = mysql.connector.connect(
  host="sql12.freesqldatabase.com",
  user="sql12605513",
  password="qPVHlJBcbK",
  database="sql12605513"
)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM iris_mysql")

myresult = mycursor.fetchall()
mycursor.close()
df_mysql = pd.DataFrame(myresult, columns=['ID','petallength'])
df_mysql = df_mysql.drop('ID', axis=1)
df_mysql


,petallength
0,1.4
1,1.4
2,1.3
3,1.5
4,1.4
...,...
145,5.2
146,5.0
147,5.2
148,5.4


In [ ]:
# Menggabungkan data dari setiap database yg telah diakses sebelumnya
df = pd.concat([df_postgreSQL, df_elephant,df_mysql, df_sqlServer], axis=1)
df

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
#memisah data untuk X dan Y
X = df.values[:, :4]
Y = df.values[:, 4]
print(X)

[[5.1 3.5 1.4 0.2]
 [4.9 3.0 1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.0 3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.0 3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.0 1.4 0.1]
 [4.3 3.0 1.1 0.1]
 [5.8 4.0 1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.0 0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.0 3.0 1.6 0.2]
 [5.0 3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.0 3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.1 1.5 0.1]
 [4.4 3.0 1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.0 3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.0 3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.0 1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.0 3.3 1.4 0.2]
 [7.0 3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [ ]:
#membagi data training dan data testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)

**Mengimplementasikan metode Desicion Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
#mendefine variable y prediksi
y_pred = classifier.predict(X_test)

**Menghitung nilai akurasi**

In [ ]:
#menghitung nilai akurasi Decision tree
from sklearn.metrics import accuracy_score
print (" DecisionTree Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

 DecisionTree Accuracy :  96.66666666666667


**Mengimplementasikan metode KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
#mendefine variable y prediksi
y_pred = classifier.predict(X_test)

**Menghitung akurasi**

In [ ]:
print (" KNN Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

 KNN Accuracy :  100.0
